### Speech Emotion Classifier 

In [1]:
datasetpath = ''
try:
  datasetpath=("NewDataset/")
  print("Folder path located!")
except FileNotFoundError:
  print("Error locating the path!")


Folder path located!


### Importing Necessary Libraries 

In [2]:
import librosa
import soundfile as sf
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

### Listing The Subdirectories of my dataset

In [3]:
os.listdir(datasetpath)

['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

### Feature Extraction 

##### The below below function extracts mfcc features from the sound file

In [4]:
def extract_feature(file_name, mfcc, chroma, mel):
    with sf.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    
    return result

### Labelling The Emotions of the Sample

In [5]:

def findEmotion(filename): 
      if filename[:1] == "a" :
          return "Anger"
      
      elif filename[:1] == "f" :
          return "Fear"
        
      elif filename[:1] == "h" :
          return "Happy"
        
      elif filename[:1] == "n" :
          return "Neutral"
        
      elif filename[:1] == "s" :
          if filename[:2] == "sa" :
              return "Sad"              
          else : 
              return "Surprised"
      else : 
          return "Disgust"

### Loading The Dataset for Feature Extraction

In [6]:
subs = os.listdir(datasetpath)
x,y=[],[]
def load_samples(test_size=0.2):
  audiosamples = []
  emotion = []
  for sub in  subs: 
    for soundfile in glob.glob(datasetpath + sub + "/*.wav"):      
      file_name=os.path.basename(soundfile)
      emotion = findEmotion(file_name)
      audiosamples.append(file_name)
      feature = extract_feature(soundfile, mfcc=True, chroma=True, mel=True)
      x.append(feature)
      y.append(emotion)
  return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

### Calling the Function For Feature Extraction 

### A look into our classifier classes 

In [7]:
set(y)

set()

<hr>

## ML Model 

This segment of the file works on building an ML model using various classifier and testing the accuracy. 

In [8]:
x_train,x_test,y_train,y_test=load_samples(test_size=0.25)

In [28]:
import pandas as pd
x_train
pd.DataFrame(x_train).to_csv('features.csv')
pd.DataFrame(y_train).to_csv('emotionclasses.csv')

In [29]:
# reading two csv files
data1 = pd.read_csv('features.csv')
data2 = pd.read_csv('emotionclasses.csv')
  
# using merge function by setting how='inner'
output1 = data1.join(data2, lsuffix="Features", rsuffix="Emotionclasses")
  
# displaying result
print(output1)

pd.DataFrame(output1).to_csv('df.csv')

     Unnamed: 0Features   0Features           1          2          3  \
0                     0 -302.414612  107.463562   7.931349  31.805050   
1                     1 -449.374939  133.274170  25.462761  35.235252   
2                     2 -446.050995  139.719437  38.104706  24.253126   
3                     3 -395.017517   93.677269  16.260586  34.029671   
4                     4 -610.346680  124.946060  23.505457  24.790319   
..                  ...         ...         ...        ...        ...   
355                 355 -596.079285  133.538742  32.895496  27.255610   
356                 356 -376.958344   81.351189   7.455905  37.361153   
357                 357 -338.724823  119.830879  28.849352  23.962114   
358                 358 -439.126495  123.436203  25.393318  38.707325   
359                 359 -460.370300  115.886818  34.241615  40.091694   

             4          5         6          7          8  ...           172  \
0    18.983475  -8.241529 -3.613688  -4.850

In [10]:
x_test.shape

(120, 180)

#### Checking the shape of the training and testing datasets

In [11]:
print((x_train.shape[0], x_test.shape[0]))

(360, 120)


#### Features of the sound samples

In [12]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


### Multi Layer Perceptron Classifier

In [13]:
model=MLPClassifier(alpha=0.01, 
                    batch_size=256, 
                    epsilon=1e-08, 
                    hidden_layer_sizes=(300,), 
                    learning_rate='adaptive', 
                    max_iter=500)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy found through MLP Classifier is : {:.2f}%".format(accuracy*100))

Accuracy found through MLP Classifier is : 62.50%


#### SVM Classifier 

In [14]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
svmclf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svmclf.fit(x_train,y_train)
y_pred=svmclf.predict(x_test)
svmaccuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy found through SVM Classifier is : {:.2f}%".format(svmaccuracy*100))

Accuracy found through SVM Classifier is : 61.67%


### Random Forest Classifier 

In [15]:
from sklearn.ensemble import RandomForestClassifier
randomForestclf = RandomForestClassifier(n_estimators = 100) 
randomForestclf.fit(x_train, y_train)
y_pred = randomForestclf.predict(x_test)
randomforestaccuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy found through SVM Classifier is : {:.2f}%".format(randomforestaccuracy*100))

Accuracy found through SVM Classifier is : 66.67%


### Naive Bayes Classifier 

In [16]:
from sklearn.naive_bayes import GaussianNB
nbclassifier = GaussianNB()
nbclassifier.fit(x_train, y_train)
y_pred  = nbclassifier.predict(x_test)
nbclassifieraccuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy found through Naive Bayes Classifier is : {:.2f}%".format(nbclassifieraccuracy*100))

Accuracy found through Naive Bayes Classifier is : 47.50%


<hr>

###### Here we can see that Random Forest Algorithm Provides the Best Accuracy Result 

<br>

## Exporting The Model

In [17]:
import pickle
modelname = "speechemotionclassifier"
pickle.dump(randomForestclf,open(modelname,"wb"))